In [47]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
from timeit import default_timer as timer

In [3]:
data=pd.read_csv('30_features-1.csv', index_col=0)

In [4]:
data.head()

,fulladdress_count_30,address_count_14,fulladdress_pct_increase_0_to_30,fulladdress_count_0_by_30,address_count_3,fulladdress_count_0_by_7,fulladdress_pct_increase_0_to_7,address_count_1,fulladdress_count_1,fulladdress_homephone_count_30,...,ssn_name_dob_count_14,ssn_firstname_count_0_by_30,ssn_firstname_pct_increase_0_to_30,ssn_lastname_count_0_by_30,fulladdress_homephone_pct_increase_0_to_30,fulladdress_homephone_count_0_by_30,address_pct_increase_1_to_14,dob_count_wSame_homephone_7,name_count_wSame_homephone_7,fraud_label
0,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,2,2,0
1,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,2,2,1
2,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,1,1,0
3,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,1,1,0
4,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,1,1,0


In [6]:
data.shape

(1000000, 31)

In [18]:
Model=data.iloc[38511:833507,].copy()
OOT=data.iloc[833507:,].copy()

In [74]:
### 30 vars
X_oot=OOT.iloc[:,:30].values
y_oot=OOT.loc[:,'fraud_label'].values
X=Model.iloc[:,:30].values
y=Model.loc[:,'fraud_label'].values
stdsc.fit(X)
X=stdsc.transform(X)
X_oot=stdsc.transform(X_oot)

In [75]:
def NN_model(classifier,X,y,X_oot,y_oot):
    start = timer()
    N=10
    FDR_train_30=np.zeros(N)
    FDR_test_30=np.zeros(N)
    FDR_oot_30=np.zeros(N)
    i=0
    kfold=StratifiedKFold(n_splits=10,shuffle=True)
    for train, test in kfold.split(X, y):
        X_train, X_test, y_train, y_test= X[train], X[test], y[train], y[test]
        classifier.fit(X_train, y_train)
        y_pred_tr=classifier.predict_proba(X_train)[:,1]
        y_pred_ts=classifier.predict_proba(X_test)[:,1]
        y_pred_oot=classifier.predict_proba(X_oot)[:,1]

        train_temp=pd.DataFrame(columns=['y_train','y_pred_tr'])
        Rows=round(len(y_train)*0.03)
        train_temp['y_train']=y_train
        train_temp['y_pred_tr']=y_pred_tr
        train_temp.sort_values('y_pred_tr', ascending=False, inplace=True)
        tempR=train_temp.head(Rows)
        num=tempR.loc[:,'y_train']
        den=((y_train==1).sum())
        fdr_30_tr=sum(num)/den
        FDR_train_30[i]=fdr_30_tr

        test_temp=pd.DataFrame(columns=['y_test','y_pred_ts'])
        Rows=round(len(y_test)*0.03)
        test_temp['y_test']=y_test
        test_temp['y_pred_ts']=y_pred_ts
        test_temp.sort_values('y_pred_ts', ascending=False, inplace=True)
        tempR=test_temp.head(Rows)
        num=tempR.loc[:,'y_test']
        den=((y_test==1).sum())
        fdr_30_ts=sum(num)/den
        FDR_test_30[i]=fdr_30_ts

        oot_temp=pd.DataFrame(columns=['y_oot','y_pred_oot'])
        Rows=round(len(y_oot)*0.03)
        oot_temp['y_oot']=y_oot
        oot_temp['y_pred_oot']=y_pred_oot
        oot_temp.sort_values('y_pred_oot', ascending=False, inplace=True)
        tempR=oot_temp.head(Rows)
        num=tempR.loc[:,'y_oot']
        den=((y_oot==1).sum())
        fdr_30_oot=sum(num)/den
        FDR_oot_30[i]=fdr_30_oot
        i = i+1
    
    print(f'train:{FDR_train_30.mean()}')
    print(f'test:{FDR_test_30.mean()}')
    print(f'oot:{FDR_oot_30.mean()}')
    print(f'Total run time{(timer()-start)/60:0.2f}min')

In [ ]:
classifier=MLPClassifier(hidden_layer_sizes=(12,),max_iter=100)
NN_model(classifier,X,y,X_oot,y_oot)

In [ ]:
classifier=MLPClassifier(hidden_layer_sizes=(15,),max_iter=100)
NN_model(classifier,X,y,X_oot,y_oot)

In [ ]:
classifier=MLPClassifier(hidden_layer_sizes=(15,),max_iter=200)
NN_model(classifier,X,y,X_oot,y_oot)

In [ ]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=150)
NN_model(classifier,X,y,X_oot,y_oot)

In [ ]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=150, solver='sgd')
NN_model(classifier,X,y,X_oot,y_oot)

In [ ]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=150, alpha=0.001)
NN_model(classifier,X,y,X_oot,y_oot)

In [ ]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=150, activation='logistic')
NN_model(classifier,X,y,X_oot,y_oot)

In [ ]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=150, activation='logistic',alpha=0.001)
NN_model(classifier,X,y,X_oot,y_oot)

In [35]:
classifier=MLPClassifier(hidden_layer_sizes=(25,),max_iter=150)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5803394019679573
test:0.580016405721685
oot:0.562070410729254
Total run time6.57min


In [36]:
classifier=MLPClassifier(hidden_layer_sizes=(25,),max_iter=250)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5801266088851366
test:0.5791459320784927
oot:0.5622799664710814
Total run time6.74min


In [37]:
classifier=MLPClassifier(hidden_layer_sizes=(30,),max_iter=250)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5804458457657351
test:0.5798418864457201
oot:0.5621961441743503
Total run time6.91min


In [38]:
classifier=MLPClassifier(hidden_layer_sizes=(30,),max_iter=300)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5805619017893656
test:0.5794936818260388
oot:0.562699077954736
Total run time7.03min


In [39]:
classifier=MLPClassifier(hidden_layer_sizes=(25,),max_iter=300)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5801072833703084
test:0.5781009391593356
oot:0.5627409891031014
Total run time6.50min


In [40]:
classifier=MLPClassifier(hidden_layer_sizes=(40,),max_iter=400)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5804264865631994
test:0.5797547784317828
oot:0.5616512992455993
Total run time8.04min


In [41]:
classifier=MLPClassifier(hidden_layer_sizes=(50,),max_iter=500)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5803491180642415
test:0.5788844564126358
oot:0.5623218776194469
Total run time8.51min


In [42]:
classifier=MLPClassifier(hidden_layer_sizes=(30,),max_iter=500)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5799621529836275
test:0.5792330400924299
oot:0.5618608549874267
Total run time7.27min


In [43]:
classifier=MLPClassifier(hidden_layer_sizes=(25,),max_iter=500)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5801846135027107
test:0.5791465385746732
oot:0.5621123218776194
Total run time8.04min


In [49]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=300, verbose=True)
NN_model(classifier,X,y,X_oot,y_oot)

Iteration 1, loss = 0.05980822
Iteration 2, loss = 0.04428900
Iteration 3, loss = 0.04379711
Iteration 4, loss = 0.04359291
Iteration 5, loss = 0.04348942
Iteration 6, loss = 0.04338955
Iteration 7, loss = 0.04325757
Iteration 8, loss = 0.04323483
Iteration 9, loss = 0.04323260
Iteration 10, loss = 0.04316759
Iteration 11, loss = 0.04311321
Iteration 12, loss = 0.04310620
Iteration 13, loss = 0.04306179
Iteration 14, loss = 0.04306386
Iteration 15, loss = 0.04302857
Iteration 16, loss = 0.04305109
Iteration 17, loss = 0.04301656
Iteration 18, loss = 0.04297428
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.06181533
Iteration 2, loss = 0.04378699
Iteration 3, loss = 0.04344146
Iteration 4, loss = 0.04329490
Iteration 5, loss = 0.04317470
Iteration 6, loss = 0.04309739
Iteration 7, loss = 0.04301472
Iteration 8, loss = 0.04297256
Iteration 9, loss = 0.04294830
Iteration 10, loss = 0.04289613
Iteration 11, loss = 0.04290267


In [50]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=300, activation='logistic')
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5800395233541249
test:0.5805382956850829
oot:0.5614836546521376
Total run time5.95min


In [51]:
classifier=MLPClassifier(hidden_layer_sizes=(25,),max_iter=400, verbose=True)
NN_model(classifier,X,y,X_oot,y_oot)

Iteration 1, loss = 0.05977611
Iteration 2, loss = 0.04415106
Iteration 3, loss = 0.04366785
Iteration 4, loss = 0.04351772
Iteration 5, loss = 0.04333532
Iteration 6, loss = 0.04323363
Iteration 7, loss = 0.04315876
Iteration 8, loss = 0.04308271
Iteration 9, loss = 0.04306139
Iteration 10, loss = 0.04300787
Iteration 11, loss = 0.04297558
Iteration 12, loss = 0.04292481
Iteration 13, loss = 0.04291303
Iteration 14, loss = 0.04291219
Iteration 15, loss = 0.04288338
Iteration 16, loss = 0.04288271
Iteration 17, loss = 0.04285607
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.06061379
Iteration 2, loss = 0.04391386
Iteration 3, loss = 0.04346750
Iteration 4, loss = 0.04327071
Iteration 5, loss = 0.04314662
Iteration 6, loss = 0.04307200
Iteration 7, loss = 0.04294209
Iteration 8, loss = 0.04286776
Iteration 9, loss = 0.04283455
Iteration 10, loss = 0.04276027
Iteration 11, loss = 0.04274485
Iteration 12, loss = 0.04271575


In [52]:
classifier=MLPClassifier(hidden_layer_sizes=(25,),max_iter=400, activation='logistic')
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5799331244732905
test:0.5788837741044326
oot:0.5608968985750209
Total run time5.81min


In [53]:
classifier=MLPClassifier(hidden_layer_sizes=(30,),max_iter=500, verbose=True)
NN_model(classifier,X,y,X_oot,y_oot)

Iteration 1, loss = 0.05670596
Iteration 2, loss = 0.04390501
Iteration 3, loss = 0.04357649
Iteration 4, loss = 0.04344503
Iteration 5, loss = 0.04331160
Iteration 6, loss = 0.04323474
Iteration 7, loss = 0.04318646
Iteration 8, loss = 0.04312283
Iteration 9, loss = 0.04308309
Iteration 10, loss = 0.04304643
Iteration 11, loss = 0.04299109
Iteration 12, loss = 0.04294676
Iteration 13, loss = 0.04294650
Iteration 14, loss = 0.04289376
Iteration 15, loss = 0.04291677
Iteration 16, loss = 0.04288648
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.05541095
Iteration 2, loss = 0.04363404
Iteration 3, loss = 0.04323805
Iteration 4, loss = 0.04313232
Iteration 5, loss = 0.04302893
Iteration 6, loss = 0.04294210
Iteration 7, loss = 0.04287358
Iteration 8, loss = 0.04284802
Iteration 9, loss = 0.04279123
Iteration 10, loss = 0.04273862
Iteration 11, loss = 0.04275710
Iteration 12, loss = 0.04268806
Iteration 13, loss = 0.04270218


In [54]:
classifier=MLPClassifier(hidden_layer_sizes=(30,),max_iter=500, activation='logistic')
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5802330058944317
test:0.5799291460837026
oot:0.5606035205364627
Total run time6.11min


In [64]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=300, activation='logistic',solver='sgd')
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5626850435356536
test:0.5627752355479541
oot:0.5371332774518022
Total run time8.32min


In [65]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=150, activation='logistic', alpha=0.001)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.57960421922066
test:0.5787964386544275
oot:0.5598072087175189
Total run time5.72min


In [66]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=300, activation='logistic', alpha=0.001)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5793333672456256
test:0.5792321303481591
oot:0.5593880972338642
Total run time5.63min


In [67]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=300, activation='logistic')
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5798267255924558
test:0.5796672913577325
oot:0.5613998323554066
Total run time5.49min


In [69]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=300, activation='logistic')
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5801943164982197
test:0.5794931511418806
oot:0.5613998323554065
Total run time6.08min


In [70]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=300)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5796332720610081
test:0.5793194657981642
oot:0.5611064543168484
Total run time6.78min


In [44]:
### 25 vars
X_oot=OOT.iloc[:,:25].values
y_oot=OOT.loc[:,'fraud_label'].values
X=Model.iloc[:,:25].values
y=Model.loc[:,'fraud_label'].values
stdsc.fit(X)
X=stdsc.transform(X)
X_oot=stdsc.transform(X_oot)

In [45]:
classifier=MLPClassifier(hidden_layer_sizes=(25,),max_iter=300)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5387913183708631
test:0.5382292737511485
oot:0.525859178541492
Total run time6.75min


In [46]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=300)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5386848970315572
test:0.5378808416953993
oot:0.5258172673931266
Total run time6.41min


In [56]:
classifier=MLPClassifier(hidden_layer_sizes=(15,),max_iter=250)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5386074845514257
test:0.5378810691314672
oot:0.52476948868399
Total run time5.52min


In [57]:
classifier=MLPClassifier(hidden_layer_sizes=(15,),max_iter=250, activation='logistic')
NN_model(classifier,X,y,X_oot,y_oot)

train:0.538897703215153
test:0.5376201241497682
oot:0.5259010896898576
Total run time5.29min


In [58]:
classifier=MLPClassifier(hidden_layer_sizes=(20,),max_iter=300, activation='logistic')
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5393620311801068
test:0.537794112741575
oot:0.5256915339480301
Total run time5.31min


In [59]:
classifier=MLPClassifier(hidden_layer_sizes=(25,),max_iter=400)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5388686793836639
test:0.5377072321637055
oot:0.5259849119865884
Total run time6.18min


In [60]:
classifier=MLPClassifier(hidden_layer_sizes=(25,),max_iter=400, activation='logistic')
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5390524739007763
test:0.5377942643656202
oot:0.5255658005029338
Total run time5.47min


In [61]:
classifier=MLPClassifier(hidden_layer_sizes=(30,),max_iter=500)
NN_model(classifier,X,y,X_oot,y_oot)

train:0.538955750878131
test:0.5370972486300768
oot:0.5251886001676446
Total run time6.58min


In [62]:
classifier=MLPClassifier(hidden_layer_sizes=(30,),max_iter=500, activation='logistic')
NN_model(classifier,X,y,X_oot,y_oot)

train:0.5393813857037941
test:0.5385777058068977
oot:0.5258172673931265
Total run time5.57min
